This is an IPython notebook containing my NOEMA integration time calculations. Currently it focuses on observations of our five Perseus protostars.

**Summary**: I can hit the four remaining protostars at **H13CN 3-2 with ~10 sigma** sensitivity in a **total of 2.5 hours _*including overhead*_**.
In addition, if we wanted to observe the **H13CN 1-0** line toward each star, the total time would be **less than 2 hours**, dominated by overhead -- the on-source time is less than 2 minutes total.

In [130]:
"""
A simple exposure-time calculator based in an IPython Notebook.

Equations and terminology derive directly from 
http://www.iram.fr/GENERAL/calls/w16/NOEMACapabilities.pdf
"IRAM NOEMA interferometer: Observing Capabilities and Current Status",
particularly Section 2.5: Signal to Noise.

"""

from __future__ import division

import numpy as np

import astropy.units as u
import astropy.constants as c

### Signal-to-noise.
#   The rms noise for a point source can be computed from
#
#   sigma = ( J_pK * T_sys / (eta * np.sqrt(N_a * (N_a -1) * T_ON * B)) *
#             1 / np.sqrt(N_pol) )
#
#   where
#   * J_pK is the conversion factor from Kelvin to Jansky (22 Jy/K in band 1,
#     29 Jy/K in band 2, and 35 Jy/K in band 3.

J_pK = {'1': 22 * u.Jy/u.K, 
        '2': 29 * u.Jy/u.K,
        '3': 35 * u.Jy/u.K}

#   * eta is an additional efficiency factor due to atmospheric phase noise.
#     These factors take into account average phase stability in typical winter conditions.

eta = {'1': 0.9,
       '2': 0.85,
       '3': 0.8}

#   * T_sys is the system temperature. For typical winter conditions:

def T_sys(frequency):
    if frequency <= 110 * u.GHz:
        return 90 * u.K
    elif frequency <= 116 * u.GHz:
        return 175 * u.K
    elif frequency <= 150 * u.GHz:
        return 130 * u.K
    elif frequency <= 178 * u.GHz:
        return 170 * u.K
    else: 
        return 200 * u.K
#   These values apply for sources at elevation >= 20 deg.

#   * N_a is the number of antennas: currently 8 when using WideX 
#     but N_a = 6 for spectral resolutions better than 2 MHz with  
#     the narrow-band correlator.

N_a = 8

#   * T_ON is the on-source integration time in seconds: 2 to 8 
#     hours, depending on source declination. Because of various
#     calibration overheads, the total observing time is typically
#     1.6 T_ON.

#   * B is the spectral bandwidth in Hz: 40 kHz to 1.25 MHz for 
#     spectral line observations using the narrow-band correlator,
#     and from 2 MHz up to 3.6 GHz when using WideX.
B = 2 * u.MHz

#   * N_pol is the number of polarizations: 1 for single-
#     polarization and 2 for dual polarization.
N_pol = 2

def band_from_frequency(frequency):
    nu = frequency
    if (nu >= 76.5*u.GHz) and (nu <= 116*u.GHz):
        return '1'
    elif (nu >= 130*u.GHz) and (nu <= 178*u.GHz):
        return '2'
    elif (nu >= 202*u.GHz) and (nu <= 274*u.GHz):
        return '3'
    else:
        raise ValueError("Frequency is not in a valid band (76.5-116, 130-178, or 202-274 GHz).")

def sigma_from_integration_time(T_ON, frequency=None, B=B):
    nu = u.Quantity(frequency, u.GHz)
    
    band = band_from_frequency(nu)
    
    sigma = ( J_pK[band] * T_sys(nu) / (eta[band] * np.sqrt(N_a * (N_a-1) * T_ON * B)) *
              1 / np.sqrt(N_pol) )
    
    return sigma.to(u.Jy)

result = sigma_from_integration_time(1*u.hr, frequency=259*u.GHz)
print result.to(u.mJy)

9.74389944472 mJy


In [131]:
print 'Test the capabilities of my calculator against the reference values from'
print 'section 2.4.1 of "An Introduction to the IRAM NOEMA interferometer"'
print ""


sigma_90_test = sigma_from_integration_time(1*u.hr, frequency=90*u.GHz, B=3.6*u.GHz)
print "  @ 90 GHz"
print "Calculated: {0:.3f}    Expected: 0.06 mJy".format(sigma_90_test.to(u.mJy))
sigma_150_test = sigma_from_integration_time(1*u.hr, frequency=150*u.GHz, B=3.6*u.GHz)
print "  @ 150 GHz "
print "Calculated: {0:.3f}    Expected: 0.12 mJy".format(sigma_150_test.to(u.mJy))
sigma_230_test = sigma_from_integration_time(1*u.hr, frequency=230*u.GHz, B=3.6*u.GHz)
print "  @ 150 GHz "
print "Calculated: {0:.3f}    Expected: 0.23 mJy".format(sigma_230_test.to(u.mJy))

Test the capabilities of my calculator against the reference values from
section 2.4.1 of "An Introduction to the IRAM NOEMA interferometer"

  @ 90 GHz
Calculated: 0.058 mJy    Expected: 0.06 mJy
  @ 150 GHz 
Calculated: 0.116 mJy    Expected: 0.12 mJy
  @ 150 GHz 
Calculated: 0.230 mJy    Expected: 0.23 mJy


In [132]:
# These are the 3-2 fluxes for the five protostars
flux_array_3_2 = np.array([192,
                          247,
                          164,
                          208,
                          371]) * u.mJy * 0.8

print "S/N reached in 1 hour for each source:"
for x in (flux_array_3_2 / result).decompose():
    print "{0:.1f},".format(x),
print ""
print ""

# time_list_3_2 = np.array([0.27, 0.16, 0.36, 0.22, 0]) * u.hr
time_list_3_2 = (10 / (flux_array_3_2 / result).decompose())**2 * u.hr

print "List of integration times needed to hit 10-sigma on the H13CN J=3-2 line:"
for x in time_list_3_2:
    print "{0:.2f},".format(x),
print ""

time_list_3_2[4] = 0
sensitivity_array = u.Quantity([sigma_from_integration_time(x, frequency=259*u.GHz) for x in time_list_3_2])

print ""
print "Total (on-source) time for 3-2 observations: {0:.3f}".format(np.sum(time_list_3_2))
print "Time for 3-2 observations with overheads: {0:.3f}".format(np.sum(time_list_3_2)*1.6)

S/N reached in 1 hour for each source:
15.8, 20.3, 13.5, 17.1, 30.5, 

List of integration times needed to hit 10-sigma on the H13CN J=3-2 line:
0.40 h, 0.24 h, 0.55 h, 0.34 h, 0.11 h, 

Total (on-source) time for 3-2 observations: 1.540 h
Time for 3-2 observations with overheads: 2.464 h


In [133]:
freq_3mm = 86.340 * u.GHz

# one hour at 86 GHz gives us what sensitivity?
result_3mm = sigma_from_integration_time(1*u.hr, frequency=freq_3mm)
print result_3mm.to(u.mJy)

2.44989471753 mJy


In [136]:
# These are the 1-0 fluxes for the five protostars
flux_array_1_0 = np.array([1.74,
                          1.19,
                          1.07,
                          0.80,
                          0.90]) * u.Jy * 0.8 

time_list_1_0 = (20 / (flux_array_1_0 / result_3mm).decompose())**2 * u.hr

print time_list_1_0
print time_list_1_0.to(u.s)

sensitivity_array_3mm = u.Quantity([sigma_from_integration_time(x, frequency=freq_3mm) for x in time_list_1_0])

print ""
print "Total (on-source) time for 1-0 observations: {0:.3f}".format(np.sum(time_list_1_0.to(u.min)))
print "Time for 1-0 observations with overheads: {0:.3f}".format(np.sum(time_list_1_0.to(u.min))*1.6)

[ 0.00123901  0.00264899  0.00327648  0.00586131  0.00463116] h
[  4.46045194   9.53637758  11.79532211  21.10072545  16.67217813] s

Total (on-source) time for 1-0 observations: 1.059 min
Time for 1-0 observations with overheads: 1.695 min
